# Evaluation of Vabamorf's disambiguation on spellchecker's suggestions

In this tiny experiment, we use Vabamorf's spellchecker to add _**multiple normalized forms** to the **words layer**_, and examine, how this increased ambiguity affects the quality of Vabamorf's disambiguation in EstNLTK.

We use EstNLTK's version 1.6.4beta (from the commit [52c921eb3d](https://github.com/estnltk/estnltk/tree/52c921eb3d06ebc0976c0dac84bc9b9f72b0491e)), and evaluate tools on the Estonian Web Treebank (EWTB) corpus.

### Load Estonian Web Treebank corpus

You can download the UD format EWTB corpus from here: https://github.com/UniversalDependencies/UD_Estonian-EWT/ (exact commit: [6cd4d14](https://github.com/UniversalDependencies/UD_Estonian-EWT/tree/6cd4d1480c1f3dc89bcdddab56f04dc51bfa8b48)).

In [1]:
eval_data_dir = 'UD_Estonian-EWT-master'

import os, os.path
from ewtb_ud_utils import load_EWTB_ud_file_with_corrections

# Load corpus files with corrections
ud_layer_name = 'ud_syntax'
loaded_texts  = []
for fname in os.listdir( eval_data_dir ):
    if fname.endswith('.conllu'):
        fpath = os.path.join( eval_data_dir, fname )
        text = load_EWTB_ud_file_with_corrections( fpath, ud_layer_name )
        text.meta['file'] = fname
        loaded_texts.append( text )

In [2]:
# get training part of the corpus
training_text = [text for text in loaded_texts if 'train' in text.meta['file']][0]

---

## Vabamorf's analysis and disambiguation (baseline: no spelling suggestions)

### 0. No spelling suggestions + Vabamorf's analysis

In [3]:
# 'morph_0' == VabamorfAnalyzer + PostMorphAnalysisTagger
from estnltk.taggers import VabamorfAnalyzer, PostMorphAnalysisTagger

vm_analyser = VabamorfAnalyzer(output_layer='morph_0')
post_corrector = PostMorphAnalysisTagger(output_layer='morph_0')
for text in loaded_texts:
    vm_analyser.tag( text )
    post_corrector.retag( text )

In [4]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_0', ud_layer_name, 'morph_0_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### 1. No spelling suggestions + Vabamorf's analysis with disambiguation

In [5]:
# 'morph_1' == VabamorfTagger
from estnltk.taggers import VabamorfTagger

vm_tagger = VabamorfTagger(output_layer='morph_1',
                           input_words_layer='words')
for text in loaded_texts:
    vm_tagger.tag( text )

In [6]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_1', ud_layer_name, 'morph_1_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### Results: Vabamorf's analysis and disambiguation (baseline: no spelling suggestions)

In [7]:
morph_0_diff = training_text.morph_0_diff_layer
morph_1_diff = training_text.morph_1_diff_layer
correct_analyses = 0
words_to_disambiguate = 0
correctly_disambiguated = 0
correct_analyses_total_after_disamb = 0
correct_analyses_total_before_disamb = 0
disambiguated = 0
for word_1, word_2 in zip(morph_0_diff, morph_1_diff):
    if word_1.annotations[0]['full_match']:
        # Look only words that obtained a full match:
        # 1) with the default morphological analysis
        correct_analyses_total_before_disamb += 1
        if (len(word_1.annotations) > 1):
            words_to_disambiguate += 1
            if len(word_2.annotations) == 1:
                disambiguated += 1
                if word_2.annotations[0]['full_match']:
                    correctly_disambiguated += 1
            if word_2.annotations[0]['full_match']:
                correct_analyses += 1
        if word_2.annotations[0]['full_match']:
            correct_analyses_total_after_disamb += 1
print('='*80)
print( ' Words that needed disambiguation:               ', words_to_disambiguate, '/', len(morph_0_diff) )
print( '   Incorrectly disambiguated:                    ', (disambiguated-correctly_disambiguated), '/', words_to_disambiguate, '   {:.02f}%'.format(((disambiguated-correctly_disambiguated)/words_to_disambiguate)*100.0) )
print( '   Correctly disambiguated:                      ', correctly_disambiguated, '/', words_to_disambiguate, '   {:.02f}%'.format((correctly_disambiguated/words_to_disambiguate)*100.0) )
print( '   Disambiguation attempts:                      ', disambiguated, '/', words_to_disambiguate, '   {:.02f}%'.format((disambiguated/words_to_disambiguate)*100.0) )
print()
print( '   Correct words (including undisambiguated):    ', correct_analyses, '/', words_to_disambiguate, '   {:.02f}%'.format((correct_analyses/words_to_disambiguate)*100.0) )
print('='*80)
print( ' VM words alignable to UD morph words (before disamb): ', correct_analyses_total_before_disamb,'/', len(morph_0_diff), '   {:.02f}%'.format((correct_analyses_total_before_disamb/len(morph_0_diff))*100.0) )
print( ' VM words alignable to UD morph words  (after disamb): ', correct_analyses_total_after_disamb,'/', len(morph_0_diff), '   {:.02f}%'.format((correct_analyses_total_after_disamb/len(morph_0_diff))*100.0) )

 Words that needed disambiguation:                6226 / 17181
   Incorrectly disambiguated:                     541 / 6226    8.69%
   Correctly disambiguated:                       4200 / 6226    67.46%
   Disambiguation attempts:                       4741 / 6226    76.15%

   Correct words (including undisambiguated):     5680 / 6226    91.23%
 VM words alignable to UD morph words (before disamb):  16664 / 17181    96.99%
 VM words alignable to UD morph words  (after disamb):  16118 / 17181    93.81%


---

## Vabamorf's analysis and disambiguation with spelling suggestions

### Reload the data

In [8]:
eval_data_dir = 'UD_Estonian-EWT-master'

import os, os.path
from ewtb_ud_utils import load_EWTB_ud_file_with_corrections

# Load corpus files with corrections
ud_layer_name = 'ud_syntax'
loaded_texts  = []
for fname in os.listdir( eval_data_dir ):
    if fname.endswith('.conllu'):
        fpath = os.path.join( eval_data_dir, fname )
        text = load_EWTB_ud_file_with_corrections( fpath, ud_layer_name )
        text.meta['file'] = fname
        loaded_texts.append( text )

In [9]:
# get training part of the corpus
training_text = [text for text in loaded_texts if 'train' in text.meta['file']][0]

### Create VMSpellingSuggestionsTagger

Make a tagger that creates a special words layer containing spelling suggestions:

In [10]:
from estnltk import Text, Annotation, ElementaryBaseSpan
from estnltk.layer.layer import Layer
from estnltk.taggers import Tagger
from estnltk.vabamorf import morf as vm
from estnltk.taggers.morph_analysis.morf_common import _get_word_texts

class VMSpellingSuggestionsTagger(Tagger):
    '''Creates normalized_words layer which contains spelling suggestions from Vabamorf's spellchecker.'''
    conf_param = []
    output_attributes = []
    
    def __init__(self, words_layer='words', output_layer='normalized_words'):
        self.input_layers = [words_layer]
        self.output_layer = output_layer
        self.output_attributes = ('normalized_form',)
    
    def _make_layer(self, text, layers, status):
        normalzed_words = Layer(name=self.output_layer,
                                attributes=self.output_attributes,
                                text_object=text,
                                ambiguous=True)
        words_layer = layers[self.input_layers[0]]
        for word in words_layer:
            if 'normalized_form' in words_layer.attributes:
                word_texts = _get_word_texts(word)
            else:
                word_texts = [word.text]
            suggestions = set()
            for word_text in word_texts:
                spell_check_result = vm.spellcheck([word_text], suggestions=True)
                # Check if we have a misspelled word with suggestions
                for item in spell_check_result:
                    if not item["spelling"] and len(item["suggestions"]) > 0:
                        for new_suggestion in item["suggestions"]:
                            if new_suggestion not in suggestions:
                                suggestions.add( new_suggestion )                
            if suggestions:
                for suggestion in suggestions:
                    normalzed_words.add_annotation( word.base_span, normalized_form=suggestion )
            else:
                normalzed_words.add_annotation( word.base_span, normalized_form=None )
        return normalzed_words


test_text = Text('Ma tahax teada assju.')
test_text.tag_layer(['words'])
VMSpellingSuggestionsTagger().tag(test_text).normalized_words

Layer(name='normalized_words', attributes=('normalized_form',), spans=SL[Span('Ma', [{'normalized_form': None}]),
Span('tahax', [{'normalized_form': 'tahad'}, {'normalized_form': 'tahaks'}, {'normalized_form': 'taha'}]),
Span('teada', [{'normalized_form': None}]),
Span('assju', [{'normalized_form': 'assjõu'}, {'normalized_form': 'asju'}]),
Span('.', [{'normalized_form': None}])])

### Apply VMSpellingSuggestionsTagger on the input corpus

In [11]:
spelling_suggestor = VMSpellingSuggestionsTagger()
for text in loaded_texts:
    spelling_suggestor.tag( text )

### 0. No spelling suggestions + Vabamorf's analysis

In [12]:
from estnltk.taggers import VabamorfAnalyzer, PostMorphAnalysisTagger

vm_analyser = VabamorfAnalyzer(output_layer='morph_0')
post_corrector = PostMorphAnalysisTagger(output_layer='morph_0')
for text in loaded_texts:
    vm_analyser.tag( text )
    post_corrector.retag( text )

In [13]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_0', ud_layer_name, 'morph_0_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### 1. Spelling suggestions + Vabamorf's analysis only

In [14]:
# 'morph_1' == VabamorfAnalyzer + PostMorphAnalysisTagger
from estnltk.taggers import VabamorfAnalyzer, PostMorphAnalysisTagger

vm_analyser = VabamorfAnalyzer(output_layer='morph_1',
                               input_words_layer=spelling_suggestor.output_layer)
post_corrector = PostMorphAnalysisTagger(output_layer='morph_1')
for text in loaded_texts:
    vm_analyser.tag( text )
    post_corrector.retag( text )

In [15]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_1', ud_layer_name, 'morph_1_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### 2. Spelling suggestions + Vabamorf's analysis with disambiguation

In [16]:
# 'morph_2' == VabamorfTagger
from estnltk.taggers import VabamorfTagger

vm_tagger = VabamorfTagger(output_layer='morph_2',
                           input_words_layer=spelling_suggestor.output_layer)
for text in loaded_texts:
    vm_tagger.tag( text )

In [17]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_2', ud_layer_name, 'morph_2_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### Results: Vabamorf's analysis and disambiguation after spelling suggestions

In [18]:
assert len(training_text.morph_0_diff_layer) == len(training_text.morph_1_diff_layer)
assert len(training_text.morph_1_diff_layer) == len(training_text.morph_2_diff_layer)

In [19]:
morph_0_diff = training_text.morph_0_diff_layer
morph_1_diff = training_text.morph_1_diff_layer
morph_2_diff = training_text.morph_2_diff_layer
correct_analyses = 0
words_to_disambiguate = 0
correctly_disambiguated = 0
correct_analyses_total_after_disamb = 0
correct_analyses_total_before_disamb = 0
disambiguated = 0
for word_1, word_2, word_3 in zip(morph_0_diff, morph_1_diff, morph_2_diff):
    if word_1.annotations[0]['full_match'] or word_2.annotations[0]['full_match']:
        # Look only words that obtained a full match:
        # 1) with the default morphological analysis, or
        # 2) with the extended morphological analysis based on normalized word forms
        correct_analyses_total_before_disamb += 1
        if (len(word_1.annotations) > 1 or len(word_2.annotations) > 1):
            words_to_disambiguate += 1
            if len(word_3.annotations) == 1:
                disambiguated += 1
                if word_3.annotations[0]['full_match']:
                    correctly_disambiguated += 1
            if word_3.annotations[0]['full_match']:
                correct_analyses += 1
        if word_3.annotations[0]['full_match']:
            correct_analyses_total_after_disamb += 1
print('='*80)
print( ' Words that needed disambiguation:               ', words_to_disambiguate, '/', len(morph_0_diff) )
print( '   Incorrectly disambiguated:                    ', (disambiguated-correctly_disambiguated), '/', words_to_disambiguate, '   {:.02f}%'.format(((disambiguated-correctly_disambiguated)/words_to_disambiguate)*100.0) )
print( '   Correctly disambiguated:                      ', correctly_disambiguated, '/', words_to_disambiguate, '   {:.02f}%'.format((correctly_disambiguated/words_to_disambiguate)*100.0) )
print( '   Disambiguation attempts:                      ', disambiguated, '/', words_to_disambiguate, '   {:.02f}%'.format((disambiguated/words_to_disambiguate)*100.0) )
print()
print( '   Correct words (including undisambiguated):    ', correct_analyses, '/', words_to_disambiguate, '   {:.02f}%'.format((correct_analyses/words_to_disambiguate)*100.0) )
print('='*80)
print( ' VM words alignable to UD morph words (before disamb): ', correct_analyses_total_before_disamb,'/', len(morph_0_diff), '   {:.02f}%'.format((correct_analyses_total_before_disamb/len(morph_0_diff))*100.0) )
print( ' VM words alignable to UD morph words  (after disamb): ', correct_analyses_total_after_disamb,'/', len(morph_0_diff), '   {:.02f}%'.format((correct_analyses_total_after_disamb/len(morph_0_diff))*100.0) )

 Words that needed disambiguation:                6370 / 17181
   Incorrectly disambiguated:                     658 / 6370    10.33%
   Correctly disambiguated:                       4199 / 6370    65.92%
   Disambiguation attempts:                       4857 / 6370    76.25%

   Correct words (including undisambiguated):     5701 / 6370    89.50%
 VM words alignable to UD morph words (before disamb):  16820 / 17181    97.90%
 VM words alignable to UD morph words  (after disamb):  16111 / 17181    93.77%


In [20]:
# Fine-grained analysis: how much of the normalized words were actually correctly disambiguated?
from estnltk.taggers.morph_analysis.morf_common import _get_word_texts
norm_words   = training_text[spelling_suggestor.output_layer]
morph_1_diff = training_text.morph_1_diff_layer
morph_2_diff = training_text.morph_2_diff_layer
words_with_normalization = 0
words_with_normalization_aligned_before_disamb = 0
words_with_normalization_aligned_after_disamb  = 0
for norm_word, word_2, word_3 in zip(norm_words, morph_1_diff, morph_2_diff):
    w_texts = _get_word_texts(norm_word)
    if len(w_texts) > 1 or (len(w_texts) == 1 and w_texts[0] != norm_word.text):
        words_with_normalization += 1
        if word_2.annotations[0]['full_match']:
            words_with_normalization_aligned_before_disamb += 1
        if word_3.annotations[0]['full_match']:
            words_with_normalization_aligned_after_disamb += 1
print('='*80)
print( ' Only words with normalizations (spelling suggestions): ', words_with_normalization, '/', len(morph_1_diff) )
print( '   Correctly analysed (without disambiguation):         ', words_with_normalization_aligned_before_disamb, '/', words_with_normalization, '   {:.02f}%'.format((words_with_normalization_aligned_before_disamb/words_with_normalization)*100.0) )
print( '       - Correctly disambiguated:                       ', words_with_normalization_aligned_after_disamb, '/', words_with_normalization, '   {:.02f}%'.format((words_with_normalization_aligned_after_disamb/words_with_normalization)*100.0) )
incorrect = words_with_normalization_aligned_before_disamb - words_with_normalization_aligned_after_disamb
print( '       - Incorrectly disambiguated:                     ', incorrect, '/', words_with_normalization, '   {:.02f}%'.format((incorrect/words_with_normalization)*100.0) )
norm_words_no_ud_match = words_with_normalization - words_with_normalization_aligned_before_disamb
print( '   Word\'s analyses cannot be matched to UD word\'s:      ', norm_words_no_ud_match, '/', words_with_normalization, '   {:.02f}%'.format((norm_words_no_ud_match/words_with_normalization)*100.0) )
print()

 Only words with normalizations (spelling suggestions):  409 / 17181
   Correctly analysed (without disambiguation):          162 / 409    39.61%
       - Correctly disambiguated:                        131 / 409    32.03%
       - Incorrectly disambiguated:                      31 / 409    7.58%
   Word's analyses cannot be matched to UD word's:       247 / 409    60.39%



---

## Summary

    All measurements made on the training part of the EWTB corpus
    
    A) Vabamorf's analysis & disambiguation (baseline: no spelling corrections)
    ================================================================================
     Words that needed disambiguation:                6226 / 17181
       Incorrectly disambiguated:                     541 / 6226      8.69%
       Correctly disambiguated:                       4200 / 6226    67.46%
       Disambiguation attempts:                       4741 / 6226    76.15%

       Correct words (including undisambiguated):     5680 / 6226    91.23%
    ================================================================================
     VM words alignable to UD morph words (before disamb):  16664 / 17181    96.99%
     VM words alignable to UD morph words  (after disamb):  16118 / 17181    93.81%

    B) Vabamorf's analysis & disambiguation on words with spelling suggestions
    ================================================================================
     Words that needed disambiguation:                6372 / 17181
       Incorrectly disambiguated:                     660 / 6372     10.36%
       Correctly disambiguated:                       4199 / 6372    65.90%
       Disambiguation attempts:                       4859 / 6372    76.26%

       Correct words (including undisambiguated):     5701 / 6372    89.47%
    ================================================================================
     VM words alignable to UD morph words (before disamb):  16820 / 17181    97.90%
     VM words alignable to UD morph words  (after disamb):  16103 / 17181    93.73%


     ================================================================================
     Only words with normalizations (spelling suggestions):  409 / 17181
       Correctly analysed (without disambiguation):          162 / 409    39.61%
           - Correctly disambiguated:                        131 / 409    32.03%
           - Incorrectly disambiguated:                      31 / 409      7.58%
       Word's analyses cannot be matched to UD word's:       247 / 409    60.39%
     ================================================================================
     